# CLIP推理（效果不好）

In [1]:
from PIL import Image
import requests
from transformers import CLIPProcessor, CLIPModel
model = CLIPModel.from_pretrained("C:/Users/LiuFeng/.cache/huggingface/hub/models--openai--clip-vit-base-patch16/snapshots/57c216476eefef5ab752ec549e440a49ae4ae5f3")
processor = CLIPProcessor.from_pretrained("C:/Users/LiuFeng/.cache/huggingface/hub/models--openai--clip-vit-base-patch16/snapshots/57c216476eefef5ab752ec549e440a49ae4ae5f3")

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)
inputs = processor(text=["a photo of a cat", "a photo of a dog"], images=image, return_tensors="pt", padding=True)

outputs = model(**inputs)
logits_per_image = outputs.logits_per_image # this is the image-text similarity score
logits_per_image.softmax(dim=1) # we can take the softmax to get the label probabilities

e:\tech\极客时间\训练营\A7-多模态大模型训练营2599\作业\MultiModelTrain\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
W1206 22:19:11.752000 5940 Lib\site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


tensor([[9.9916e-01, 8.4369e-04]], grad_fn=<SoftmaxBackward0>)

In [2]:
import pandas as pd
test = pd.read_csv("data/re_id_1000_test.txt", header=None)

In [3]:
class_names = ["Toyota_Camry", "Toyota_Corolla", "Toyota_Corolla_EX", "Buick_LaCrosse", "Volkswagen_Magotan", "Audi_A4",
               "Nissan_Sylphy", "Nissan_Tiida", "Honda_Accord", "Ford_Focus"]

In [4]:
image = Image.open("data/image/" + test[0].values[100].replace("\\", "/"))
inputs = processor(text=["a photo of a " + x for x in class_names], images=image, return_tensors="pt", padding=True)
outputs = model(**inputs)
logits_per_image = outputs.logits_per_image
logits_per_image.softmax(dim=1)

tensor([[0.1830, 0.1186, 0.1731, 0.0035, 0.0034, 0.0037, 0.3545, 0.1169, 0.0231,
         0.0201]], grad_fn=<SoftmaxBackward0>)

# CNN

In [1]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import models, transforms, datasets
from transformers import TrainingArguments, Trainer
import pandas as pd
from PIL import Image
import numpy as np
import warnings
warnings.filterwarnings("ignore")

import evaluate
accuracy_metric = evaluate.load("accuracy")

e:\tech\极客时间\训练营\A7-多模态大模型训练营2599\作业\MultiModelTrain\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
W1206 22:50:52.173000 24712 Lib\site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


In [9]:
import pandas as pd
test = pd.read_csv("data/re_id_1000_test.txt", header=None)
train = pd.read_csv("data/re_id_1000_train.txt", header=None)

test["label"] = test[0].apply(lambda x: int(x.split("\\")[0]) - 1)
test["path"] = test[0].apply(lambda x: "data/image/" + x.replace("\\", "/"))

train["label"] = train[0].apply(lambda x: int(x.split("\\")[0]) - 1)
train["path"] = train[0].apply(lambda x: "data/image/" + x.replace("\\", "/"))

In [10]:
test.dtypes

0        object
label     int64
path     object
dtype: object

In [11]:
train.head(3)

,0,label,path
0,1\License_1\4404000000002940408492.jpg,0,data/image/1/License_1/4404000000002940408492.jpg
1,1\License_1\4404000000002949435646.jpg,0,data/image/1/License_1/4404000000002949435646.jpg
2,1\License_1\4404000000002949812715.jpg,0,data/image/1/License_1/4404000000002949812715.jpg


In [12]:
class CustomImageDataset(Dataset):
    def __init__(self, dataframe, unique_labels, transform=None):
        self.dataframe = dataframe
        self.transform = transform
        self.unique_labels = unique_labels
        self.label_to_index = {label: i for i, label in enumerate(self.unique_labels)}
        self.num_classes = len(self.label_to_index)

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx]['path']
        try:
            image = Image.open(img_path).convert('RGB')
        except FileNotFoundError:
            print(f"Error: Image not found at {img_path}. Returning a dummy sample.")
            return torch.zeros(3, 224, 224), 0

        label_name = self.dataframe.iloc[idx]['label']
        label_index = self.label_to_index[label_name]

        if self.transform:
            image = self.transform(image)

        return {'pixel_values': image, 'labels': label_index}

In [13]:
def get_data_transforms():
    return transforms.Compose([
        transforms.Resize(256),         # Resize shortest side to 256
        transforms.CenterCrop(224),     # Center crop to 224x224 (ResNet input size)
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

def custom_collate_fn(batch):
    batch = [item for item in batch if item is not None]
    pixel_values = torch.stack([item['pixel_values'] for item in batch])
    labels = torch.tensor([item['labels'] for item in batch], dtype=torch.long)
    
    return {'pixel_values': pixel_values, 'labels': labels}

class ResNetClassifier(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        
        self.model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
        num_ftrs = self.model.fc.in_features
        self.model.fc = nn.Linear(num_ftrs, num_classes)
        self.criterion = nn.CrossEntropyLoss()

    def forward(self, pixel_values, labels=None):
        logits = self.model(pixel_values)
        if labels is not None:
            loss = self.criterion(logits, labels)
            return (loss, logits)
        
        return logits

# ✅ 用 evaluate 替代 load_metric
import evaluate
accuracy_metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    return accuracy_metric.compute(predictions=predictions, references=labels)

In [14]:
batch_size = 64

transform = get_data_transforms()

train_set = CustomImageDataset(train, train["label"].unique(), transform)
test_set = CustomImageDataset(test, train["label"].unique(), transform)

train_len = int(0.9 * len(train_set))
val_len = len(train_set) - train_len
train_set, val_set = random_split(train_set, [train_len, val_len])

num_classes = 10 
model = ResNetClassifier(num_classes=num_classes)

In [15]:
training_args = TrainingArguments(
    output_dir='./results',                          # Output directory for checkpoints and logs
    num_train_epochs=1,                              # Total number of training epochs
    per_device_train_batch_size=batch_size,          # Batch size per device during training
    per_device_eval_batch_size=batch_size,           # Batch size for evaluation
    warmup_steps=500,                                # Number of warmup steps for learning rate scheduler
    weight_decay=0.01,                               # Strength of weight decay
    logging_dir='./logs',                            # Directory for storing logs
    logging_steps=50,
    evaluation_strategy="epoch",                     # Evaluate after each epoch
    save_strategy="epoch",                           # Save checkpoint after each epoch
    load_best_model_at_end=True,                     # Load the best model at the end
    metric_for_best_model="accuracy",                # Use accuracy to determine best model
    learning_rate=5e-4,                              # Starting learning rate
    remove_unused_columns=False,                     # Keep all columns in the dataset for flexibility
    dataloader_num_workers=0,
)

trainer = Trainer(
    model=model,                                     # The instantiated model
    args=training_args,                              # Training arguments
    train_dataset=train_set,                         # Training dataset
    eval_dataset=val_set,                            # Validation dataset
    compute_metrics=compute_metrics,                 # Function to compute evaluation metrics
    data_collator=custom_collate_fn,
)

trainer.train()

                                      
  0%|          | 0/71 [04:20<?, ?it/s]         

{'loss': 1.3393, 'grad_norm': 1.3046152591705322, 'learning_rate': 5e-05, 'epoch': 0.7}










                                      

                                       
  0%|          | 0/71 [05:22<?, ?it/s]         

                                      
100%|██████████| 71/71 [03:15<00:00,  2.75s/it]

{'eval_loss': 0.009686819277703762, 'eval_accuracy': 1.0, 'eval_runtime': 10.6867, 'eval_samples_per_second': 46.787, 'eval_steps_per_second': 0.749, 'epoch': 1.0}
{'train_runtime': 195.2534, 'train_samples_per_second': 23.047, 'train_steps_per_second': 0.364, 'train_loss': 0.9618983789229058, 'epoch': 1.0}


TrainOutput(global_step=71, training_loss=0.9618983789229058, metrics={'train_runtime': 195.2534, 'train_samples_per_second': 23.047, 'train_steps_per_second': 0.364, 'total_flos': 0.0, 'train_loss': 0.9618983789229058, 'epoch': 1.0})

In [16]:
test_results = trainer.evaluate(test_set)
test_results

100%|██████████| 79/79 [01:51<00:00,  1.41s/it]


{'eval_loss': 0.015545063652098179,
 'eval_accuracy': 0.999,
 'eval_runtime': 113.5685,
 'eval_samples_per_second': 44.026,
 'eval_steps_per_second': 0.696,
 'epoch': 1.0}

In [17]:
def predict_single_image(model, image_path, transform, label_names):
    try:
        image = Image.open(image_path).convert('RGB')
        input_tensor = transform(image)
        input_batch = input_tensor.unsqueeze(0) 

        model.eval()
        device = next(model.parameters()).device
        input_batch = input_batch.to(device)

        with torch.no_grad():
            output = model(input_batch)

        probabilities = torch.nn.functional.softmax(output, dim=1)
        probabilities = probabilities[0]
        predicted_index = torch.argmax(probabilities).item()
        predicted_label = label_names[predicted_index]
        confidence = probabilities[predicted_index].item() * 100

        print(f"Predicted Class Index: {predicted_index}")
        print(f"Predicted Label: {predicted_label}")
        print(f"Confidence: {confidence:.2f}%")

    except FileNotFoundError:
        print(f"ERROR: Image file not found at path: {image_path}")
    except Exception as e:
        import traceback
        traceback.print_exc()
        print(f"An unexpected error occurred during prediction: {e}")

In [18]:
train.sample(3)

,0,label,path
1501,4\License_301\4404000000002941735050.jpg,3,data/image/4/License_301/440400000000294173505...
2586,6\License_518\4404000000002942172377.jpg,5,data/image/6/License_518/440400000000294217237...
2653,6\License_531\4404000000002947342645.jpg,5,data/image/6/License_531/440400000000294734264...


In [20]:
class_names = ["Toyota_Camry", "Toyota_Corolla", "Toyota_Corolla_EX", "Buick_LaCrosse", "Volkswagen_Magotan", "Audi_A4",
               "Nissan_Sylphy", "Nissan_Tiida", "Honda_Accord", "Ford_Focus"]
predict_single_image(trainer.model, "data/image/1/License_20/4404000000002940725419.jpg", transform, class_names)

Predicted Class Index: 0
Predicted Label: Toyota_Camry
Confidence: 99.73%


In [21]:
class_names = ["Toyota_Camry", "Toyota_Corolla", "Toyota_Corolla_EX", "Buick_LaCrosse", "Volkswagen_Magotan", "Audi_A4",
               "Nissan_Sylphy", "Nissan_Tiida", "Honda_Accord", "Ford_Focus"]
predict_single_image(trainer.model, "data/image/6/License_531/4404000000002947607633.jpg", transform, class_names)

Predicted Class Index: 5
Predicted Label: Audi_A4
Confidence: 99.45%


In [22]:
import transformers
print(transformers.__version__)

4.41.2
